In [1]:
import tensorflow as tf
from tensorflow import keras as keras
print(tf.__version__)

print(keras.__version__)

2.12.0
2.12.0


In [2]:
import matplotlib.pyplot as plt
import os
import sys
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.python.keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Dense, Dropout, Embedding, SimpleRNN, LSTM, Bidirectional
from tensorflow.keras.callbacks import History

# Modelowanie szeregów czasowych przy użyciu sieci rekurencyjnych

https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/

In [3]:
import pandas
import matplotlib.pyplot as plt
dataframe = pandas.read_csv('Dane/monthly-car-sales-in-quebec-1960.csv', usecols=[1], engine='python', skipfooter=3)
plt.plot(dataframe)
plt.show()

FileNotFoundError: ignored

# Zad
Wykonaj predykcję szeregu czasowego.

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA


time_series = dataframe

# Przygotowanie danych
train_data = time_series[:-3]  # Dane treningowe (bez ostatnich 3 obserwacji)
test_data = time_series[-3:]   # Dane testowe (ostatnie 3 obserwacje)

# Tworzenie i trenowanie modelu ARIMA
model = ARIMA(train_data, order=(1, 0, 0))  # Tworzenie modelu ARIMA z parametrami (p, d, q)
model_fit = model.fit()

# Wykonanie predykcji dla danych testowych
predictions = model_fit.forecast(steps=len(test_data))[0]

# Wyświetlanie wyników predykcji
print("Predykcje: ", predictions)